Question 1 - The original stores dataset only has the addresses of the stores. To find out the distance a driver travels while on a route, we would need to find the latitude and longitude of these stores. And follow this up by calculating the total distance covered after travelling to each of the stores on the route. The following code is used for this purpose - 

In [0]:
import numpy as np
import pandas as pd
import requests
import xlwt
from tempfile import TemporaryFile

In [0]:
storedf = pd.read_csv("C:\\Users\\asus\\Desktop\\anycsvfilewithaddress.csv")
lat = []
lng = []
for i in range(len(storedf) - 1):
    jrq = "https://maps.googleapis.com/maps/api/geocode/json?address=" + str(storedf["Address"][i+1])+"&key=API KEY"
    rsp = requests.get(jrq)
    jsonpl = rsp.json()
    if jsonpl['results'] == []:
        latitude = 0.00
        longitude = 0.00
        lat.append(latitude)
        lng.append(longitude)
        continue
    latitude = jsonpl['results'][0]['geometry']['location']['lat']
    longitude = jsonpl['results'][0]['geometry']['location']['lng']
    lat.append(latitude)
    lng.append(longitude)

In [0]:
wb = xlwt.Workbook()
res = wb.add_sheet('res')
for i,e in enumerate(lat):
    res.write(i,1,e)

for i,e in enumerate(lng):
    res.write(i,2,e)

name = "destinationfile.xls"
wb.save(name)
wb.save(TemporaryFile())

In [0]:
def reverselist(x):
	x = [i for i in reversed(x)]
	return x

delivery_df = pd.read_csv("The route data file")
sv_df = pd.read_csv("The store visit data file")
store_df = pd.read_csv("The store data file")

def gdist(x1,y1,x2,y2):
	org = (x1,y1)
	des = (x2,y2)
 # API = 	#Put API key here
  result = googlemaps.Client(key=API).distance_matrix(org, des, mode='driving')["rows"][0]["elements"][0]["distance"]["value"]
  return result

ID = []
for i in range(len(delivery_df)-1):
	# print(i)
	id = delivery_df["ID"][i]
	ID.append(int(id))

finaldistance = []
for i in ID:
	x = sv_df.index[sv_df['RID'] == str(i)].tolist()
	
	stores = []
	for j in x:
		stores.append((sv_df.iloc[j]['Store ID']))
	
	distance = []
	for k in range(len(stores) - 1):
		x1 = store_df.iloc[int(store_df[store_df['Store ID']==stores[k]].index[0])]['Lat']
		y1 = store_df.iloc[int(store_df[store_df['Store ID']==stores[k]].index[0])]['Long']
		x2 = store_df.iloc[int(store_df[store_df['Store ID']==stores[k + 1]].index[0])]['Lat']
		y2 = store_df.iloc[int(store_df[store_df['Store ID']==stores[k + 1]].index[0])]['Long']
		distance.append(gdist(x1,y1,x2,y2))
	finaldistance.append(sum(distance))



In [0]:
wb = xlwt.Workbook()
res = wb.add_sheet('res')
for i,e in enumerate(finaldistance):
    res.write(i,1,e)

name = "destinationfile.xls"
wb.save(name)
wb.save(TemporaryFile())

After this we would like to calculate an experience score for the loaders, assemblers and driver. The following code calculates the score - 


In [0]:
def potential(x):
	potential = (10*x)/70
	return potential

def increxp(x):
	x += 0.5
	return x

delivery_df = pd.read_csv("C:\\Users\\asus\\Desktop\\amltest\\route test.csv")

dexp = []
lexp = []
aexp = []
for h in delivery_df['DRIVERID']:
	a = (delivery_df.DRIVERID == i).sum()
	lexp.append(a)
 
for i in delivery_df['LOADERID']:
	a = (delivery_df.LOADERID == i).sum()
	lexp.append(a)

for j in delivery_df['ASSEMBLERID']:
	a = (delivery_df.ASSEMBLERID == j).sum()
	aexp.append(a)

dexp = list(map(int, dexp))
lexp = list(map(int, lexp))
aexp = list(map(int, aexp))


In [0]:
wb = xlwt.Workbook()
res = wb.add_sheet('res')
for i,e in enumerate(dexp):
    res.write(i,1,e)
for i,e in enumerate(aexp):
    res.write(i,2,e)
for i,e in enumerate(lexp):
    res.write(i,3,e)
name = "destinationfile.xls"
wb.save(name)
wb.save(TemporaryFile())

The above code calculates the number of times a driver, loader or assembler has been on a trip. This data is added to the route data as DRIVEREXP, LOADEREXP and ASSEMBLEREXP. The following code gives a score based on their experience and potential 

In [0]:
checkexp = delivery_df['DRIVEREXP'].tolist()
lexp = delivery_df['LOADEREXP'].tolist()
aexp = delivery_df['ASSEMBLEREXP'].tolist()
checkexp = list(map(int, checkexp))
lexp = list(map(int, lexp))
aexp = list(map(int, aexp))
revcheckexp = reverselist(checkexp)
revlexp = reverselist(lexp)
revaexp = reverselist(aexp)
driverexperience = {}
loaderexperience = {}
assemblerexperience = {}
for i in range(len(lexp)):
	loaderexperience[lexp[i]] = potential(delivery_df['LOADEREXP'][i])
	assemblerexperience[aexp[i]] = potential(delivery_df['ASSEMBLEREXP'][i])
	driverexperience[checkexp[i]] = potential(delivery_df['DRIVEREXP'][i])

updateexp = []
for i in revcheckexp:
	update = increxp(driverexperience[i])
	updateexp.append(update)
	driverexperience[i] = update

updateexp = reverselist(updateexp)

updatelexp = []
for i in revlexp:
	# print(driverexperience[i])
	update = increxp(loaderexperience[i])
	# print(update)
	updatelexp.append(update)
	loaderexperience[i] = update

updatelexp = reverselist(updatelexp)

updateaexp = []
for i in revaexp:
	update = increxp(assemblerexperience[i])
	updateaexp.append(update)
	assemblerexperience[i] = update

updateaexp = reverselist(updateaexp)

In [0]:
wb = xlwt.Workbook()
res = wb.add_sheet('res')
for i,e in enumerate(updateexp):
    res.write(i,1,e)
for i,e in enumerate(updateaexp):
    res.write(i,2,e)
for i,e in enumerate(updatelexp):
    res.write(i,3,e)
name = "destinationfile.xls"
wb.save(name)
wb.save(TemporaryFile())

We also need to find out at what time of the day did the route start. We used floor function in excel to round of the exact start time to the nearest hour. The following code puts this data into categories

In [0]:
x = delivery_df["Floor"].between(0,7).tolist()
y = delivery_df["Floor"].between(8,20).tolist()
z = delivery_df["Floor"].between(21,23).tolist()

x1 = [1 if i == True else 0 for i in x]
y1 = [1 if i == True else 0 for i in y]
z1 = [1 if i == True else 0 for i in z]

In [0]:
wb = xlwt.Workbook()
res = wb.add_sheet('res')
for i,e in enumerate(x1):
    res.write(i,1,e)
for i,e in enumerate(y1):
    res.write(i,2,e)
for i,e in enumerate(z1):
    res.write(i,3,e)
name = "destinationfile.xls"
wb.save(name)
wb.save(TemporaryFile())

Now we have to calculate the time that it took to complete the delivery. The time would be rounded off to the nearest hour and will be the target value of our dataset

In [0]:
from datetime import datetime
df['Time Taken'] = df['Actual End'] - df['Actual Start']

In [0]:
from datetime import datetime
import math

end = pd.to_datetime(df['Actual End'])
start = pd.to_datetime(df['Actual Start'])

diff = end - start
diff_list = list(diff)
a= []
b= []
for _ in range(len(diff_list)):
    print(_)
    temp = diff_list[_]
    temp_in_s = temp.total_seconds()
    minutes = divmod(temp_in_s, 60)[0]
    a.append(minutes)
    hour = minutes / 60
    print(hour)
    hours = math.ceil(hour)
    b.append(hours)
    
    
    
    
df['Time taken (in Minutes)'] = a  
df['Time taken (in Hours)'] = b

In [0]:
export_csv = df.to_csv (r'destination file', index = None, header=True)
df = pd.read_csv(r'data file') 

df['column to convert']

In [0]:
import math

a= []
for _ in range(len(m)):
    

    
    hour = m[_] / 60

    hours = math.ceil(hour)
    print(hours)
    a.append(hours)
    newdf = pd.DataFrame(a) 

export_csv = newdf.to_csv (r'destination file', index = None, header=True)

For question 2, we will be using the latitude and longitude of the stores and calculate the distance of the store from nearby warehouses. 
The code for carrying that out is - 


In [0]:
regional_df = pd.read_csv("Store Data File")
dist = []
for i in range(len(regional_df)):
	x1 = float(regional_df['Lat'][i])
	y1 = float(regional_df['Long'][i])
	x2 = #latitude of warehouse
	y2 = #longitude of warehouse
	d = gdist(x1,y1,x2,y2)
	dist.append(d)
 

In [0]:
wb = xlwt.Workbook()
res = wb.add_sheet('res')
for i,e in enumerate(dist):
    res.write(i,1,e)
name = "destinationfile.xls"
wb.save(name)
wb.save(TemporaryFile())

We will now calculate the number of stores within 200 km radius of a store associated with each warehouse.

In [0]:
ubranch = ['BRANCH1', 'BRANCH2', 'BRANCH3','BRANCH4','BRANCH5','BRANCH6','BRANCH7','BRANCH8','BRANCH9','BRANCH10']
index= []
for i in ubranch:
    x = regional_df.index[regional_df['Territory'] == i].tolist()
    index.append(x)

COUNT = []
for i in range(len(regional_df)):
    branch_index = index[8]
    x1 =  float(regional_df['Lat'][i])
    y1 =  float(regional_df['Long'][i])
    c = 0
    for j in branch_index:
        x2 = float(regional_df['Lat'][j])
        y2 = float(regional_df['Long'][j])
        distance_1 = gdist(x1,y1,x2,y2)
        if distance_1 < 200:
            c += 1
        else:
            continue
    COUNT.append(c)

In [0]:
wb = xlwt.Workbook()
res = wb.add_sheet('res')
for i,e in enumerate(COUNT):
    res.write(i,1,e)
name = "destinationfile.xls"
wb.save(name)
wb.save(TemporaryFile())

Removing outliers from the dataset for question 1

In [0]:
delivery_df = pd.read_csv("Data file")
distance = delivery_df['Distance'].tolist()
time = delivery_df['Time'].tolist()
time = delivery_df['Time'].tolist()
indices = []
lost = []
for i in range(len(distance)):
	if (25 < distance[i]/time[i] < 100):
		indices.append(i)
	else:
		lost.append(i)

for j in lost:
	delivery_df = delivery_df.drop([j], axis = 0)	


delivery_df.to_csv(r'Destination File')